In [1]:
import pandas as pd
from gbnet.models import ordinal_regression

# Data From
# https://github.com/gagolews/ordinal-regression-data

urls = [
    f"https://raw.githubusercontent.com/gagolews/ordinal-regression-data/refs/heads/master/{i}"
    for i in [
        'abalone.csv',
        'abalone_ord.csv',
        'affairs.csv',
        'ailerons.csv',
        'auto_ord.csv',
        'auto_riskness.csv',
        'bostonhousing.csv',
        'bostonhousing_ord.csv',
        'californiahousing.csv',
        'cement_strength.csv',
        'fireman_example.csv',
        'glass.csv',
        'kinematics.csv',
        'machine_ord.csv',
        'skill.csv',
        'stock_ord.csv',
        'winequality-red.csv',
        'winequality-white.csv',
        'wisconsin_breast_ord.csv'
    ]
]


In [2]:
%%time
from xgboost.sklearn import XGBClassifier, XGBRegressor
from lightgbm.sklearn import LGBMClassifier, LGBMRegressor

results = []
for url in urls:
    df = pd.read_csv(url)
    train = df.sample(int(df.shape[0] * 0.7), random_state=10110)
    test = df[~df.index.isin(train.index)]
    
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    
    xcols = [col for col in df.columns if not (col == 'response')]
    ycol = 'response'

    num_classes = df[ycol].nunique()
    num_training = train.shape[0]
    num_test = test.shape[0]

    ordboost = ordinal_regression.GBOrd(num_classes, module_type='XGBModule')
    ordboost_lgbm = ordinal_regression.GBOrd(num_classes, module_type='LGBModule')
    xgbclass = XGBClassifier(objective='multi:softmax')
    xgbmae = XGBRegressor(objective='reg:absoluteerror')
    xgbhuber = XGBRegressor(objective='reg:pseudohubererror')
    lgbclass = LGBMClassifier(objective='multiclass')
    lgbmae = LGBMRegressor(objective='mae')
    lgbhuber = LGBMRegressor(objective='huber')
    
    miny = train[ycol].min()
    ordboost.fit(train[xcols], train[ycol])
    ordboost_lgbm.fit(train[xcols], train[ycol])
    xgbclass.fit(train[xcols], train[ycol] - miny)
    xgbmae.fit(train[xcols], train[ycol])
    xgbhuber.fit(train[xcols], train[ycol])
    lgbclass.fit(train[xcols], train[ycol] - miny)
    lgbmae.fit(train[xcols], train[ycol])
    lgbhuber.fit(train[xcols], train[ycol])
    
    test['pred'] = ordboost.predict(test[xcols], return_logits=False)
    test['pred_lgbm'] = ordboost_lgbm.predict(test[xcols], return_logits=False)
    test['pred_xgbclass'] = xgbclass.predict(test[xcols]) + miny
    test['pred_xgbmae'] = xgbmae.predict(test[xcols])
    test['pred_xgbhuber'] = xgbhuber.predict(test[xcols])
    test['pred_lgbclass'] = lgbclass.predict(test[xcols]) + miny
    test['pred_lgbmae'] = lgbmae.predict(test[xcols])
    test['pred_lgbhuber'] = lgbhuber.predict(test[xcols])
    
    results.append({
        'dataset': url.split('/')[-1],
        'num_classes': num_classes,
        'training_size': num_training,
        'test_size': num_test,
        'test_mae_ord': ((test['response'] - test['pred']).abs()).mean(),
        'test_mae_lgbm': ((test['response'] - test['pred_lgbm']).abs()).mean(),
        'test_mae_xgbclass': ((test['response'] - test['pred_xgbclass']).abs()).mean(),
        'test_mae_xgbmae': ((test['response'] - test['pred_xgbmae']).abs()).mean(),
        'test_mae_xgbhuber': ((test['response'] - test['pred_xgbhuber']).abs()).mean(),
        'test_mae_lgbclass': ((test['response'] - test['pred_lgbclass']).abs()).mean(),
        'test_mae_lgbmae': ((test['response'] - test['pred_lgbmae']).abs()).mean(),
        'test_mae_lgbhuber': ((test['response'] - test['pred_lgbhuber']).abs()).mean(),
        'test_err_ord': ((test['response'] - test['pred']).abs() > 0).mean(),
        'test_err_lgbm': ((test['response'] - test['pred_lgbm']).abs() > 0).mean(),
        'test_err_xgbclass': ((test['response'] - test['pred_xgbclass']).abs() > 0).mean(),
        'test_err_xgbmae': ((test['response'] - test['pred_xgbmae'].round()).abs() > 0).mean(),
        'test_err_xgbhuber': ((test['response'] - test['pred_xgbhuber'].round()).abs() > 0).mean(),
        'test_err_lgbclass': ((test['response'] - test['pred_lgbclass']).abs() > 0).mean(),
        'test_err_lgbmae': ((test['response'] - test['pred_lgbmae'].round()).abs() > 0).mean(),
        'test_err_lgbhuber': ((test['response'] - test['pred_lgbhuber'].round()).abs() > 0).mean(),
    })    


/opt/miniconda3/envs/gbnet/lib/python3.12/site-packages/torch/autograd/graph.py:825: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1729646995093/work/torch/csrc/autograd/engine.cpp:1206.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


CPU times: user 5min 12s, sys: 4min 53s, total: 10min 5s
Wall time: 52.2 s


In [3]:
full = pd.DataFrame(results)

In [4]:
import numpy as np
def calculate_pairwise_win_rates(df, mae_columns):
    """
    Calculate pairwise win rates between MAE columns where lower values are better.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing MAE columns
    mae_columns (list): List of column names containing MAE values
    
    Returns:
    pandas.DataFrame: Matrix of win rates where entry [i,j] represents how often model i beats model j
    """
    n_models = len(mae_columns)
    win_rates = np.zeros((n_models, n_models))
    
    for i, model1 in enumerate(mae_columns):
        for j, model2 in enumerate(mae_columns):
            if i != j:
                # Count how often model1 has lower MAE than model2
                wins = (df[model1] < df[model2]).sum()
                total_comparisons = len(df)
                win_rates[i,j] = wins / total_comparisons
    
    # Create DataFrame with nice labels
    result = pd.DataFrame(
        win_rates,
        columns=[col.replace('test_mae_', '') for col in mae_columns],
        index=[col.replace('test_mae_', '') for col in mae_columns]
    )
    
    return result

In [5]:
calculate_pairwise_win_rates(
    full, [
        'test_mae_ord',
        'test_mae_lgbm',
        'test_mae_xgbclass',
        'test_mae_xgbmae',
        'test_mae_xgbhuber',
        'test_mae_lgbclass',
        'test_mae_lgbmae',
        'test_mae_lgbhuber'
    ]
).T

,ord,lgbm,xgbclass,xgbmae,xgbhuber,lgbclass,lgbmae,lgbhuber
ord,0.000000,0.631579,0.315789,0.315789,0.000000,0.315789,0.263158,0.157895
lgbm,0.263158,0.000000,0.210526,0.263158,0.052632,0.210526,0.315789,0.157895
xgbclass,0.631579,0.736842,0.000000,0.421053,0.105263,0.473684,0.473684,0.368421
xgbmae,0.684211,0.736842,0.578947,0.000000,0.000000,0.578947,0.631579,0.473684
xgbhuber,1.000000,0.947368,0.894737,1.000000,0.000000,0.947368,1.000000,0.947368
lgbclass,0.684211,0.736842,0.526316,0.421053,0.052632,0.000000,0.421053,0.421053
lgbmae,0.736842,0.684211,0.526316,0.368421,0.000000,0.578947,0.000000,0.421053
lgbhuber,0.842105,0.842105,0.631579,0.526316,0.052632,0.578947,0.578947,0.000000


In [6]:
calculate_pairwise_win_rates(
    full, [
        'test_mae_ord',
        'test_mae_lgbm',
        'test_mae_xgbclass',
        'test_mae_xgbmae',
        'test_mae_xgbhuber',
        'test_mae_lgbclass',
        'test_mae_lgbmae',
        'test_mae_lgbhuber'
    ]
).T.sum().sort_values(ascending=False)

lgbm        5.315789
ord         4.842105
xgbclass    3.684211
lgbclass    3.684211
lgbmae      3.684211
xgbmae      3.315789
lgbhuber    2.947368
xgbhuber    0.263158
dtype: float64

In [7]:
calculate_pairwise_win_rates(
    full, [
        'test_err_ord',
        'test_err_lgbm',
        'test_err_xgbclass',
        'test_err_xgbmae',
        'test_err_xgbhuber',
        'test_err_lgbclass',
        'test_err_lgbmae',
        'test_err_lgbhuber'
    ]
).T

,test_err_ord,test_err_lgbm,test_err_xgbclass,test_err_xgbmae,test_err_xgbhuber,test_err_lgbclass,test_err_lgbmae,test_err_lgbhuber
test_err_ord,0.000000,0.526316,0.473684,0.210526,0.000000,0.473684,0.210526,0.315789
test_err_lgbm,0.421053,0.000000,0.421053,0.315789,0.052632,0.473684,0.315789,0.315789
test_err_xgbclass,0.526316,0.578947,0.000000,0.421053,0.000000,0.473684,0.263158,0.473684
test_err_xgbmae,0.684211,0.684211,0.526316,0.000000,0.105263,0.578947,0.473684,0.421053
test_err_xgbhuber,1.000000,0.894737,1.000000,0.894737,0.000000,1.000000,0.947368,0.894737
test_err_lgbclass,0.473684,0.526316,0.421053,0.368421,0.000000,0.000000,0.368421,0.368421
test_err_lgbmae,0.684211,0.684211,0.736842,0.526316,0.052632,0.578947,0.000000,0.526316
test_err_lgbhuber,0.631579,0.631579,0.526316,0.526316,0.105263,0.578947,0.421053,0.000000


In [8]:
calculate_pairwise_win_rates(
    full, [
        'test_err_ord',
        'test_err_lgbm',
        'test_err_xgbclass',
        'test_err_xgbmae',
        'test_err_xgbhuber',
        'test_err_lgbclass',
        'test_err_lgbmae',
        'test_err_lgbhuber'
    ]
).T.sum().sort_values(ascending=False)

test_err_lgbm        4.526316
test_err_ord         4.421053
test_err_lgbclass    4.157895
test_err_xgbclass    4.105263
test_err_lgbhuber    3.315789
test_err_xgbmae      3.263158
test_err_lgbmae      3.000000
test_err_xgbhuber    0.315789
dtype: float64